In [1]:
import numpy as np
from scipy.stats import norm
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
import random

import arch 

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskLasso
from sklearn.linear_model import MultiTaskLassoCV

### **III)** Implementation of a method based on Lasso regression in the case of a high degree polynomial (large set if control variates) and comparison with the naive approach *(Question 3)*

**First of all, lets explain why, when the number of control variates become too large, the linear regression approach of Step 2 may become too expensive.**

First of all, when we consider polynomials, increasing by 1 the degree of the polynomial function increases a lot the number of control variates since, in a $d$-dimensionnal space, a polynomial of order $p$ provides $\binom{d+p}{d}-1$ control variates according to the first paper. 

Therefore, for instance in our example where we are in a 3-dimensional space, we effectively have $\binom{1+3}{1}-1 = \binom{4}{1}-1 = 3$ control variates that are $z_1, z_2$ and $z_3$ for the first order polynomial case. And, for the 2 order polynomial case, we would have $z_1, z_2, z_3, z_1*z_2, z_3*z_3, z_1*z_3, z_1^2, z_2^2$ and $z_3^2$ which effectively corresponds to  $\binom{2+3}{2}-1 = \binom{5}{2}-1 = 9$ control variates. In the same way, we would have $\binom{3+3}{3}-1 = \binom{6}{3}-1 = 19$ for the 3rd order polynomial case. 

On the other hand, the second paper states that the computation time of the OLS method is of the order $nm^2 + m^3 + nt$, where $n$ is the number of samples and $m$ the number of control variates. Indeed, when doing OLS, we need to compute $(𝑋′𝑋)^{−1}𝑋′𝑌$. In general, the complexity of the matrix product $AB$ of two matrices $A$ and $B$ with dimensions $a \times b$ and $b \times c$, respectively, can be expressed as $O(abc)$. Therefore, 

a) The matrix product $X'X$ has a complexity of $O(p^2n)$.

b) The matrix-vector product $X'Y$ has a complexity of $O(pn)$.

c) The inverse $(X'X)^{-1}$ has a complexity of $O(p^3)$.

Thus, the overall complexity of these operations is $O(np^2 + p^3)$.

Since we observed a rapid escalation in the number of control variates with higher polynomial degrees, this consequently escalates computational complexity when seeking optimal coefficients through OLS regression. Hence, the second paper proposes a remedy: employing Lasso regression to diminish the count of control variates utilized.

Indeed, in statistics and machine learning, LASSO (least absolute shrinkage and selection operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model.

It is similar to the OLS but with an additional term which corresponds to lambda, an hyperparameter to find, times the norm 1 of the vector of coefficients. Therefore, in a LASSO regression, we search the parameters that satisfies this : $$\displaystyle \min_{\beta \in \mathbb{R}^p} {\frac{1}{N}} |y - X\beta|_2^2 + \lambda |\beta|_1.$$

**Therefore, we can adapt the method described in the second paper as the LSLasso method in order to deal with the issue of a large set of control variates. We are going to do the second order polynomial case here.**

Indeed, the method that we are going to apply is:

**1)** We are first going to find the 9 control variates based on the derivative of ln(posterior) as we did with the 3 control variates in II° but with a second order polynomial case. We use the gradient function as in question 2 in order to compute $z_1$, $z_2$ and $z_3$, and then, the next 6 control variates that we are going to use are $z_1\times z_2$, $z_1\times z_3$, $z_2\times z_3$, $z_1^2$, $z_2^2$ and $z_3^2$. 

**2)** Then, we are going to do a LASSO Cross Validation (LassoCV) to regress the $f(w_i)$ on the vector of control variates $[z_i]$ as in Question 2 (but we have 9 control variates instead of 3 so 9 parameters to estimate for each $w_i$) in order to find the optimal hyperparameter $\lambda$.

**3)** Next, armed with the optimal hyperparameter $\lambda$, we'll conduct LASSO Regression of $f(w_i) = w_i$ on $[z_i]$, employing subsampling to lower computation time. This approach aims to zero out certain coefficients, helping us identify which control variates to retain for the final OLS regression.

**4)** With the control variates we kept (after LASSO), we are going to do our OLS regression to find the optimal parameters of the control variates.

**5)** Ultimately, we can compute, for instance, the empirical expected value (mean) of each $\tilde{f}(w_i)$ to determine if we achieve the same expected value as with only $f(w_i)$ but with reduced variance, fulfilling the objective of employing control variates.

***In this question, we are focusing on 2nd order polynomials.***
 
 As we know that for quadratic polynomials $P(x) = a^T\times x + \frac{1}{2}\times x^TBx$ (as given in the paper), the re-normalized $\tilde{f}$ is now:$\tilde{f}(x) = f(x) - \frac{1}{2}\operatorname{tr}(B) + (a + Bx)^Tz$. 
 
 Here, if we note $\alpha_1$, $\alpha_2$, $\alpha_3$, $\alpha_{12}$, $\alpha_{13}$, $\alpha_{23}$, $\alpha_{11}$, $\alpha_{22}$ and $\alpha_{33}$ the coefficients of $z_1$, $z_2$, $z_3$,$z_1\times z_2$, $z_1\times z_3$, $z_2\times z_3$, $z_1^2$, $z_2^2$ and $z_3^2$, then we have : $a = [\alpha_1, \alpha_2, \alpha_3]$ and 
$ B = \begin{pmatrix}
\alpha_{11} & \alpha_{12} & \alpha_{13} \\
\alpha_{21} & \alpha_{22} & \alpha_{23} \\
\alpha_{31} & \alpha_{32} & \alpha_{33} \\
\end{pmatrix}$ but with B symmetric so we only have 6 distinct coefficients in this matrix. 

***Step 0 :*** Let's remember the functions we'll have to use

In [2]:
def simulate_garch(T, omega_1, omega_2, omega_3):
    np.random.seed(423)  # For reproducibility
    r = np.zeros(T)
    h = np.zeros(T)
    h[0] = omega_1 / (1 - omega_2 - omega_3)
    r[0] = np.random.normal(0, np.sqrt(abs(h[0]))) # abs

    
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        r[t] = np.random.normal(0, np.sqrt(abs(h[t]))) #abs
        
    return r, h

def log_likelihood(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    h[0] = omega_1 / (1 - omega_2 - omega_3) if (omega_2 + omega_3) < 1 else 100000  # large number for stability
    
    log_lik = 0
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        log_lik -= 0.5 * (np.log(abs(h[t])) + r[t]**2 / h[t]) # abs
    
    return log_lik

def log_posterior(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    log_prior = -0.5 * (omega_1**2 / sigma_1**2 + omega_2**2 / sigma_2**2 + omega_3**2 / sigma_3**2)
    return log_likelihood(r, omega_1, omega_2, omega_3) + log_prior

def proposal(omega_old, sigma_proposal ): # function used in the MH algorithm that will "propose" the new parameter (new omega here)
    # omega_old is the list containing the old omegas, we will use a normal probability distribution to determine the new ones
    omega = np.array([0.0,0.0,0.0])
    omega[0] = np.random.normal(loc= omega_old[0], scale = sigma_proposal)
    omega[1] = np.random.normal(loc= omega_old[1], scale = sigma_proposal)
    omega[2] = np.random.normal(loc= omega_old[2], scale = sigma_proposal)

    return omega

def metropolis_hasting_algorithm(iterations, returns, sigma_proposal, initial_omega, sigma_omega):
    # Let's initialize our variable
    acceptation = np.array([False]*iterations) # we will through the iterations accept or not the change of value of omega
    omega = initial_omega
    omega_sampling = [omega]
    # Let's apply the algorithm trough all the iterations
    for t in range(iterations):
        # We propose a candidate for omegas
        omega_new = proposal(omega, sigma_proposal)
        # We calculate the acceptance alpha which is ratio of the prior of the new omega over the old omega
        alpha = log_posterior(returns,omega_new[0],omega_new[1],omega_new[2],sigma_omega[0],sigma_omega[1],sigma_omega[2]) - log_posterior(returns,omega[0],omega[1],omega[2],sigma_omega[0],sigma_omega[1],sigma_omega[2])
        # We compare alpha with 1 and an uniform random number
        u = np.random.uniform(0,1)
        if alpha > 0:  # reminder that alpha is a log here so we don't compare with 1 but with 0
            acceptation[t] = True
            omega = omega_new
        if alpha < 0:
            if alpha > np.log(u): # we compare to log u and not u
                acceptation[t]= False
                omega = omega_new # we take the new omega
        omega_sampling.append(omega)
    acceptance_rate = np.sum(acceptation)/len(acceptation)*100
    return acceptance_rate, omega_sampling



In [3]:
def compute_h_t_derivatives(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    grad_h_omega1 = np.zeros(T)
    grad_h_omega2 = np.zeros(T)
    grad_h_omega3 = np.zeros(T)

    # Initial volatility
    h[0] = omega_1  

    # Compute h and its derivatives
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        
        # Derivative of h_t with respect to omega_1
        grad_h_omega1[t] = (1 - omega_3**(t)) / (1 - omega_3) if omega_3 != 1 else t  # Using the geometric series

        # Derivative of h_t with respect to omega_2
        grad_h_omega2[t] = r[t-1]**2 + omega_3 * grad_h_omega2[t-1]

        # Derivative of h_t with respect to omega_3
        grad_h_omega3[t] = h[t-1] + omega_3 * grad_h_omega3[t-1]

    return h, grad_h_omega1, grad_h_omega2, grad_h_omega3

def compute_log_posterior_gradients(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    h, grad_h_omega1, grad_h_omega2, grad_h_omega3 = compute_h_t_derivatives(r, omega_1, omega_2, omega_3)
    T = len(r)
    gradients = np.zeros(3)

    # Compute the gradient of the log-posterior for each parameter
    gradients[0] = -omega_1 / sigma_1**2 - 0.5 * np.sum((1 / h) * grad_h_omega1 - (r**2 / h**2) * grad_h_omega1)
    gradients[1] = -omega_2 / sigma_2**2 - 0.5 * np.sum((1 / h) * grad_h_omega2 - (r**2 / h**2) * grad_h_omega2)
    gradients[2] = -omega_3 / sigma_3**2 - 0.5 * np.sum((1 / h) * grad_h_omega3 - (r**2 / h**2) * grad_h_omega3)

    return gradients

In [4]:
def compute_log_posterior_gradients_concatenator(r, sigma_1, sigma_2, sigma_3,w1_samples,w2_samples,w3_samples):
    '''
    This function computes the log posterior gradients at every points
    '''
    gradient_concat=[]
    i = 0
    for _ in r:
        gradient_concat.append(compute_log_posterior_gradients(r, w1_samples[i],w2_samples[i],w3_samples[i], sigma_1, sigma_2, sigma_3))
        i = i +1 
    return np.array(gradient_concat)

And let's define a new function that will help us calculte the 6 missing control variates

In [7]:
def list_product(x,y):
    result = []
    # The 2 lists must be of same length
    for i in range(len(x)):
        product = x[i] * y[i]
        result.append(product)
    return result

***Step 1 :*** Compute the 6 new control variates

In [41]:
# Parameters
initial_omega = [0.3, 0.3, 0.3]
true_omega = [0.6, 0.7, 0.4]  # These are the true parameters we want to estimate
sigma_omega = [9, 9, 9]  # Standard deviations for the priors

# Metropolis-Hastings settings
iterations = 5000
burn_in = 1000
sigma_proposal = 0.01

# Parameters (example values)
omega_1, omega_2, omega_3 =0.1, 0.1, 0.8
sigma_1, sigma_2, sigma_3 = 9, 9, 9
# Simulate data
T = iterations + 1 # Number of time points
r, h = simulate_garch(T, true_omega[0], true_omega[1], true_omega[2])

# Execute the Metropolis-Hastings algorithm
acceptance_rate, omega_samples = metropolis_hasting_algorithm(iterations, r, sigma_proposal, initial_omega, sigma_omega)

# We extract the parameter samples
omega_samples = np.array(omega_samples)  # Convert list of samples into an array for easier slicing
w1_samples = omega_samples[:, 0]
w2_samples = omega_samples[:, 1]
w3_samples = omega_samples[:, 2]

# We compute the log posterior gradients at every point
log_post_gradient = compute_log_posterior_gradients_concatenator(r, sigma_1, sigma_2, sigma_3, w1_samples, w2_samples, w3_samples)

# We compute the control variates (z_i)
x1 = [elem[0] if isinstance(elem, np.ndarray) else elem[0] for elem in log_post_gradient]
x2 = [elem[1] if isinstance(elem, np.ndarray) else elem[1] for elem in log_post_gradient]
x3 = [elem[2] if isinstance(elem, np.ndarray) else elem[2] for elem in log_post_gradient]

x1x2 = list_product(x1, x2)
x1x3 = list_product(x1, x3)
x2x3 = list_product(x2, x3)
x1_square = list_product(x1, x1)
x2_square = list_product(x2, x2)
x3_square = list_product(x3, x3)

# We store these 9 control variates in an array 
z = np.column_stack((x1, x2, x3, x1x2, x1x3, x2x3, x1_square, x2_square, x3_square))


In [97]:
z

array([[3.84557902e+03, 1.10675770e+04, 9.86898076e+03, ...,
        1.47884780e+07, 1.22491262e+08, 9.73967812e+07],
       [3.84557902e+03, 1.10675770e+04, 9.86898076e+03, ...,
        1.47884780e+07, 1.22491262e+08, 9.73967812e+07],
       [3.84557902e+03, 1.10675770e+04, 9.86898076e+03, ...,
        1.47884780e+07, 1.22491262e+08, 9.73967812e+07],
       ...,
       [7.10645000e+00, 3.96999555e+02, 3.83220842e+02, ...,
        5.05016316e+01, 1.57608647e+05, 1.46858214e+05],
       [7.10645000e+00, 3.96999555e+02, 3.83220842e+02, ...,
        5.05016316e+01, 1.57608647e+05, 1.46858214e+05],
       [7.10645000e+00, 3.96999555e+02, 3.83220842e+02, ...,
        5.05016316e+01, 1.57608647e+05, 1.46858214e+05]])

We indeed obtained our new z, with 9 control variates for each iteration

***Step 2 :*** Lasso cross validation to find the best hyperparameter

In [98]:
w = np.column_stack((w1_samples, w2_samples, w3_samples))

In [87]:
# We store our data in arrays after burn in 
Y = np.array(w[burn_in:])
Z = np.array(z[burn_in:])

# We also store especially w1,w2 and w3, it might be useful later
Y1 = np.array(w1_samples[burn_in:])
Y2 = np.array(w2_samples[burn_in:])
Y3 = np.array(w3_samples[burn_in:])

In [101]:
# We do a Lasso Cross Validation in order to find the optimal parameter lambda : step 2 of the method

alphas = [0.0001, 0.0001, 0.001, 0.1, 1,1.5] # List of hyperparameters that will be tested through Cross Validation

lasso_cv = LassoCV(cv=5) # évaluer les performances de la régression Lasso

lasso_cv.fit(Z,Y1) # utilisation de la validation croisée, avec Z variable de controle, Y variable expliquée

print("Optimal lambda:", lasso_cv.alpha_)

/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0007859554789024781, tolerance: 0.0006274315532178564
  model = cd_fast.enet_coordinate_descent_gram(
/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0007701482585110719, tolerance: 0.0006274315532178564
  model = cd_fast.enet_coordinate_descent_gram(
/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.000736520083183867, tolerance: 0.0006274315532178564
  model = cd_fast.enet_coordinate_descent_gram(


Optimal lambda: 0.9569567913835515


/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.319e-03, tolerance: 8.562e-04
  model = cd_fast.enet_coordinate_descent(


***Step 3 :*** Lasso regression using the parameters we found, and only using the most useful control variates.

In [106]:
# With the optimal lambda we just found, we do a Lasso regression to select the variables
lasso_bestmodel = Lasso(alpha=lasso_cv.alpha_)
lasso_bestmodel.fit(Z,Y)
coeff_lasso = lasso_bestmodel.coef_

/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.610e-03, tolerance: 8.562e-04
  model = cd_fast.enet_coordinate_descent(
/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.828e-04, tolerance: 8.884e-05
  model = cd_fast.enet_coordinate_descent(
/Users/antoine/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

# problème : ils sont quasi tous nuls

In [107]:
coeff_lasso

array([[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -3.39864865e-06, -4.07773124e-06,  8.17090660e-07,
         1.95193871e-05, -5.08005446e-07,  1.55968049e-07],
       [-0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         7.64386893e-07, -1.04728616e-06,  4.41171283e-07,
         1.74706330e-06, -7.42223873e-07,  3.04539106e-08],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         7.52965913e-08,  1.06785624e-06, -4.77198987e-07,
        -2.74421484e-06,  4.83490314e-07, -2.40883307e-08]])

We can see that some coefficients have been set to 0 by Lasso

In [108]:
def extract_non_null(coeff_lasso): # the goal is to extract the rows of the Z matrix where the coefficients of each w_i are not zero

    # Indexes of the non nul coefficients in the first, second and third column
    nonnull_w1 = np.nonzero(coeff_lasso[:, 0])[0].tolist()
    nonnull_w2 = np.nonzero(coeff_lasso[:, 1])[0].tolist()
    nonnull_w3 = np.nonzero(coeff_lasso[:, 2])[0].tolist()

    return nonnull_w1


In [109]:
extract_non_null(coeff_lasso)

[]

In [ ]:
def real_extract_non_null(coeff_lasso): # the goal is to extract the rows of the Z matrix where the coefficients of each w_i are not zero

    # Indexes of the non nul coefficients in the first, second and third column
    nonnull_w1 = np.nonzero(coeff_lasso[:, 0])[0].tolist()
    nonnull_w2 = np.nonzero(coeff_lasso[:, 1])[0].tolist()
    nonnull_w3 = np.nonzero(coeff_lasso[:, 2])[0].tolist()
    # We initialize empty lists for the 3 regressions that we are going to do of each w_i on the Z kept.
    controlvarw1 = []
    controlvarw2 = []
    controlvarw3 = []

    for i in range(len(Z)):
        # We fulfill the 3 lists with the non null values remaining of the z_i
        controlvarw1.append([Z[i,j] for j in nonnull_w1])
        controlvarw2.append([Z[i,j] for j in nonnull_w2])
        controlvarw3.append([Z[i,j] for j in nonnull_w3])
        
    return np.asarray(controlvarw1), np.asarray(controlvarw2), np.asarray(controlvarw3)


***Step 4 :*** OLS regression using the parameters we kept, and finding their optimal values

Now, let's do the 3 regressions of w_1, w_2 and w_3 on Z with the control variates we kept.

In [62]:
def OLS_reg(coeff_lasso, w1, w2, w3):
    controlvarw1, controlvarw2, controlvarw3 = real_extract_non_null(coeff_lasso)
    reg1 = LinearRegression()
    reg1.fit(controlvarw1, w1) # On fait la regression seulement sur la partie qui n'est pas nulle
    coeff_reg1 = reg1.coef_.tolist()
    reg2 = LinearRegression()
    reg2.fit(controlvarw2, w2)
    coeff_reg2 = reg2.coef_.tolist()
    reg3 = LinearRegression()
    reg3.fit(controlvarw3, w3)
    coeff_reg3 = reg3.coef_.tolist()

    # We return the coefficient of the 3 regressions
    return coeff_reg1, coeff_reg2, coeff_reg3


Now, lets create the matrix as depicted in the paper, the 3x3 matrix, dimension 9.

In [65]:

def matrix_coeff(coeff_lasso, w1, w2, w3):
    coeff_reg1 = OLS_reg(coeff_lasso, w1, w2, w3)[0]
    coeff_reg2 = OLS_reg(coeff_lasso, w1, w2, w3)[1]
    coeff_reg3 = OLS_reg(coeff_lasso, w1, w2, w3)[2]

    # We add 0 to the z_i that were removed after Lasso to have again our 9 control variates 
    # vectors with some values therefore equal to 0 if the control variate was not used
    # Thus we still are in dimension 9 while having removed the useless coefficients

    for i in np.where(coeff_lasso[:, 0] == 0)[0]:
        coeff_reg1.insert(i,0)
    for j in np.where(coeff_lasso[:, 1] == 0)[0]:
        coeff_reg2.insert(j,0)
    for l in np.where(coeff_lasso[:, 2] == 0)[0]:
        coeff_reg3.insert(l,0)
        
    #This corresponds, for each wi, to the 3 first coefficients of the control variates associated to each linear regression
    A_reg1 = [coeff_reg1[i] for i in range(3)]
    A_reg2 = [coeff_reg2[i] for i in range(3)]
    A_reg3 = [coeff_reg3[i] for i in range(3)]
    # direct effects of control variables on the parameter, constant
    
    # B is symmetric so we only need 6 coefficients for the 6 remaining coefficients of the control variates, for each of the 3 regression
    # First regression 
    reg1_B11 = coeff_reg1[6]
    reg1_B12 = coeff_reg1[3]
    reg1_B13 = coeff_reg1[5]
    reg1_B22 = coeff_reg1[7]
    reg1_B23 = coeff_reg1[4]
    reg1_B33 = coeff_reg1[8]
    B_reg1 = np.array([[reg1_B11,reg1_B12,reg1_B13],[reg1_B12,reg1_B22,reg1_B23],[reg1_B13,reg1_B23,reg1_B33]])
    # Coviarance matrix
    
    # Second regression 
    reg2_B11 = coeff_reg2[6]
    reg2_B12 = coeff_reg2[3]
    reg2_B13 = coeff_reg2[5]
    reg2_B22 = coeff_reg2[7]
    reg2_B23 = coeff_reg2[4]
    reg2_B33 = coeff_reg2[8]
    B_reg2 = np.array([[reg1_B11,reg1_B12,reg1_B13],[reg1_B12,reg1_B22,reg1_B23],[reg1_B13,reg1_B23,reg1_B33]])
    
    # Third regression 
    reg3_11 = coeff_reg3[6]
    reg3_B12 = coeff_reg3[3]
    reg3_B13 = coeff_reg3[5]
    reg3_B22 = coeff_reg3[7]
    reg3_B23 = coeff_reg3[4]
    reg3_B33 = coeff_reg3[8]
    B_reg3 = np.array([[reg1_B11,reg1_B12,reg1_B13],[reg1_B12,reg1_B22,reg1_B23],[reg1_B13,reg1_B23,reg1_B33]])
    
    return A_reg1, B_reg1, A_reg2, B_reg2, A_reg3, B_reg3

Now, we've obtained the optimal coefficients for the matrix B (symmetric so thats why we have only 6)

***Step 5 :*** Computing the values and their variance

We'll need to create a Trace function as we know the form of the polynom when it's a second order case. 
Indeed, for quadratic polynomials $P(x) = a^T\times x + \frac{1}{2}\times x^TBx$ (as given in the paper), the re-normalized $\tilde{f}$ is now:$\tilde{f}(x) = f(x) - \frac{1}{2}\operatorname{tr}(B) + (a + Bx)^Tz$. 

In [66]:
def trace(B):
    return(B[0,0] + B[1,1] + B[2,2])

Let's compute the estimators of all w now that we added control variates

In [72]:
def ZV_MCMC_Q3(coeff_lasso, w1, w2, w3):
    a_reg1, B_reg1, a_reg2, B_reg2, a_reg3, B_reg3 = matrix_coeff(coeff_lasso, w1, w2, w3)
    trace_reg1, trace_reg2, trace_reg3 = trace(B_reg1), trace(B_reg2), trace(B_reg3)
    w1_tilde = [w1[i] - (0.5)*trace_reg1 + np.dot(a_reg1 + np.dot(B_reg1,Y[i]), Z[i]) for i in range(len(Y1))]
    w2_tilde = [w2[i] - (0.5)*trace_reg2 + np.dot(a_reg2 + np.dot(B_reg2,Y[i]), Z[i]) for i in range(len(Y2))]
    w3_tilde = [w3[i] - (0.5)*trace_reg3 + np.dot(a_reg3 + np.dot(B_reg3,Y[i]), Z[i]) for i in range(len(Y3))]
    return w1_tilde, w2_tilde, w3_tilde

w1_tilde, w2_tilde, w3_tilde = ZV_MCMC_Q3(coeff_lasso, Y1, Y2, Y3)

ValueError: Found array with 0 feature(s) (shape=(4001, 0)) while a minimum of 1 is required by LinearRegression.

Let's now compare our new estimation of w with the previous ones, without the addition of control variates.

In [ ]:

print("Mean of w_1 for the ZV MCMC method with control variates =", np.mean(w1_tilde))
print("Mean of w_1 for the MCMC method without control variates =", np.mean(w1_samples[:burn_in]))
print("Variance of w_1 for the ZV MCMC method with control variates =", np.var(w1_tilde))
print("Variance of w_1 for the MCMC method without control variates =", np.var(w1_samples[:burn_in]))

print("Mean of w_2 for the ZV MCMC method with control variates =", np.mean(w2_tilde))
print("Mean of w_2 for the MCMC method without control variates =", np.mean(w2_samples[:burn_in]))
print("Variance of w_2 for the ZV MCMC method with control variates =", np.var(w2_tilde))
print("Variance of w_2 for the MCMC method with control variates =", np.var(w2_samples[:burn_in]))

print("Mean of w_3 for the ZV MCMC method with control variates =", np.mean(w3_tilde))
print("Mean of w_3 for the MCMC method with control variates =", np.mean(w3_samples[:burn_in]))
print("Variance of w_3 for the ZV MCMC method with control variates =", np.var(w3_tilde))
print("Variance of w_3 for the MCMC method with control variates =", np.var(w3_samples[:burn_in]))

It is now time for comparison with the naive method

We could compare this LASSO method with the naive method which only consists in doing the full regression of f on -z in order to find the optimal coefficients 9 components in the vector for the 2nd order polynomial case (instead of 3 in Question 2 with 1st order polynomial case).

It should lead to a lower variance but a longer time for the execution of the algorithm, which is why we would prefer use the LASSO Method when we have a high degre polynomial case.

In [ ]:
# First, we have to compute everything again


initial_omega = [0.3, 0.3, 0.3]
true_omega = [0.6, 0.7, 0.4]  # These are the true parameters we want to estimate
sigma_omega = [9, 9, 9]  # Standard deviations for the priors

# Metropolis-Hastings settings
iterations = 5000
burn_in = 1000
sigma_proposal = 0.01

# Parameters (example values)
omega_1, omega_2, omega_3 =0.1, 0.1, 0.8
sigma_1, sigma_2, sigma_3 = 9, 9, 9
# Simulate data
T = iterations + 1 # Number of time points
r, h = simulate_garch(T, true_omega[0], true_omega[1], true_omega[2])

# Execute the Metropolis-Hastings algorithm
acceptance_rate, omega_samples = metropolis_hasting_algorithm(iterations, r, sigma_proposal, initial_omega, sigma_omega)

# We extract the parameter samples
omega_samples = np.array(omega_samples)  # Convert list of samples into an array for easier slicing
w1_samples = omega_samples[:, 0]
w2_samples = omega_samples[:, 1]
w3_samples = omega_samples[:, 2]

# We compute the log posterior gradients at every point
log_post_gradient = compute_log_posterior_gradients_concatenator(r, sigma_1, sigma_2, sigma_3, w1_samples, w2_samples, w3_samples)

# We compute the control variates (z_i)
x1 = [elem[0] if isinstance(elem, np.ndarray) else elem[0] for elem in log_post_gradient]
x2 = [elem[1] if isinstance(elem, np.ndarray) else elem[1] for elem in log_post_gradient]
x3 = [elem[2] if isinstance(elem, np.ndarray) else elem[2] for elem in log_post_gradient]

x1x2 = list_product(x1, x2)
x1x3 = list_product(x1, x3)
x2x3 = list_product(x2, x3)
x1_square = list_product(x1, x1)
x2_square = list_product(x2, x2)
x3_square = list_product(x3, x3)

# We store these 9 control variates in an array 
z = np.column_stack((x1, x2, x3, x1x2, x1x3, x2x3, x1_square, x2_square, x3_square))


Y = np.array(w[burn_in:])
Z = np.array(z[burn_in:])
Y1 = np.array(w1_samples[burn_in:])
Y2 = np.array(w2_samples[burn_in:])
Y3 = np.array(w3_samples[burn_in:])
# We do the 3 regressions of w_1, w_2 and w_3 on Z : step 4 of the method 

def OLS_bis(w1, w2, w3): # New OLS function, that regresses the w_i on all Z, and not a new Z made by Lasso
    reg1 = LinearRegression()
    reg1.fit(Z, w1) # On fait la regression sur l'entiéreté de Z
    coeff_reg1 = reg1.coef_.tolist()
    reg2 = LinearRegression()
    reg2.fit(Z, w2)
    coeff_reg2 = reg2.coef_.tolist()
    reg3 = LinearRegression()
    reg3.fit(Z, w3)
    coeff_reg3 = reg3.coef_.tolist()
    # We return the coefficient of the 3 regressions
    return coeff_reg1, coeff_reg2, coeff_reg3
# We compute the a vector as well as the B matrix defined in the article and previously in the notebook

def matrix_coeff_bis(w1, w2, w3):
    coeff_reg1 = OLS_bis(w1, w2, w3)[0]
    coeff_reg2 = OLS_bis(w1, w2, w3)[1]
    coeff_reg3 = OLS_bis(w1, w2, w3)[2]
        
    #This corresponds, for each wi, to the 3 first coefficients of the control variates associated to each linear regression
    a_reg1 = [coeff_reg1[i] for i in range(3)]
    a_reg2 = [coeff_reg2[i] for i in range(3)]
    a_reg3 = [coeff_reg3[i] for i in range(3)]
    
    # B is symmetric so we only need 6 coefficients for the 6 remaining coefficients of the control variates, for each of the 3 regression
    # First regression 
    reg1_b11 = coeff_reg1[6]
    reg1_b12 = coeff_reg1[3]
    reg1_b13 = coeff_reg1[5]
    reg1_b22 = coeff_reg1[7]
    reg1_b23 = coeff_reg1[4]
    reg1_b33 = coeff_reg1[8]
    B_reg1 = np.array([[reg1_b11,reg1_b12,reg1_b13],[reg1_b12,reg1_b22,reg1_b23],[reg1_b13,reg1_b23,reg1_b33]])
    
    # Second regression 
    reg2_b11 = coeff_reg2[6]
    reg2_b12 = coeff_reg2[3]
    reg2_b13 = coeff_reg2[5]
    reg2_b22 = coeff_reg2[7]
    reg2_b23 = coeff_reg2[4]
    reg2_b33 = coeff_reg2[8]
    B_reg2 = np.array([[reg1_b11,reg1_b12,reg1_b13],[reg1_b12,reg1_b22,reg1_b23],[reg1_b13,reg1_b23,reg1_b33]])
    
    # Third regression 
    reg3_b11 = coeff_reg3[6]
    reg3_b12 = coeff_reg3[3]
    reg3_b13 = coeff_reg3[5]
    reg3_b22 = coeff_reg3[7]
    reg3_b23 = coeff_reg3[4]
    reg3_b33 = coeff_reg3[8]
    B_reg3 = np.array([[reg1_b11,reg1_b12,reg1_b13],[reg1_b12,reg1_b22,reg1_b23],[reg1_b13,reg1_b23,reg1_b33]])
    
    return a_reg1, B_reg1, a_reg2, B_reg2, a_reg3, B_reg3

def trace(B):
    return(B[0,0] + B[1,1] + B[2,2])

def ZV_MCMC_Q3_bis(w1, w2, w3):
    a_reg1, B_reg1, a_reg2, B_reg2, a_reg3, B_reg3 = matrix_coeff_bis(w1, w2, w3)
    trace_reg1, trace_reg2, trace_reg3 = trace(B_reg1), trace(B_reg2), trace(B_reg3)
    w1_tilde = [w1[i] - (0.5)*trace_reg1 + np.dot(a_reg1 + np.dot(B_reg1,Y[i]), x[i]) for i in range(len(Y1))]
    w2_tilde = [w2[i] - (0.5)*trace_reg2 + np.dot(a_reg2 + np.dot(B_reg2,Y[i]), x[i]) for i in range(len(Y2))]
    w3_tilde = [w3[i] - (0.5)*trace_reg3 + np.dot(a_reg3 + np.dot(B_reg3,Y[i]), x[i]) for i in range(len(Y3))]
    return w1_tilde, w2_tilde, w3_tilde

In [ ]:
w1_tilde_bis, w2_tilde_bis, w3_tilde_bis = ZV_MCMC_Q3_bis(Y1, Y2, Y3)

print("Mean of w_1 for the ZV MCMC method with control variates =", np.mean(w1_tilde))
print("Mean of w_1_bis for the ZV MCMC method with control variates =", np.mean(w1_tilde_bis))
print("Mean of w_1 for the MCMC method without control variates =", np.mean(w1_samples[:burn_in]))
print("Variance of w_1 for the ZV MCMC method with control variates =", np.var(w1_tilde))
print("Variance of w_1 for the ZV MCMC method with control variates =", np.var(w1_tilde_bis))
print("Variance of w_1 for the MCMC method without control variates =", np.var(w1_samples[:burn_in]))

print("Mean of w_2 for the ZV MCMC method with control variates =", np.mean(w2_tilde))
print("Mean of w_2_bis for the ZV MCMC method with control variates =", np.mean(w2_tilde_bis))
print("Mean of w_2 for the MCMC method without control variates =", np.mean(w2_samples[:burn_in]))
print("Variance of w_2 for the ZV MCMC method with control variates =", np.var(w2_tilde))
print("Variance of w_2_bis for the ZV MCMC method with control variates =", np.var(w2_tilde_bis))
print("Variance of w_2 for the MCMC method with control variates =", np.var(w2_samples[:burn_in]))

print("Mean of w_3 for the ZV MCMC method with control variates =", np.mean(w3_tilde))
print("Mean of w_3_bis for the ZV MCMC method with control variates =", np.mean(w3_tilde_bis))
print("Mean of w_3 for the MCMC method with control variates =", np.mean(w3_samples[:burn_in]))
print("Variance of w_3 for the ZV MCMC method with control variates =", np.var(w3_tilde))
print("Variance of w_3_bis for the ZV MCMC method with control variates =", np.var(w3_tilde_bis))
print("Variance of w_3 for the MCMC method with control variates =", np.var(w3_samples[:burn_in]))

Conclusion : we indeed lower the variance, but it took longer to compute